In [1]:
import jax
import jax.numpy as jnp
from jax import jit, jacfwd, lax



In [10]:
## Cross product operator (S operator)
@jit
def S(r):
    return jnp.block([[  0,       -r[2],    r[1]],
                     [r[2],     0,      -r[0]],
                     [-r[1],    r[0],      0]])

@jit
def H(r):
    return jnp.block([[jnp.identity(3), S(r).T],
                     [jnp.zeros((3, 3)), jnp.identity(3)]])

## Rotation matrices
@jit
def R_b__n(phi, theta, psi):
    x_rot = jnp.array([[1,         0,           0],
                      [0,       jnp.cos(phi),   - jnp.sin(phi)],
                      [0,       jnp.sin(phi),    jnp.cos(phi)]])
    y_rot = jnp.array([[ jnp.cos(theta),      0,        jnp.sin(theta)],
                      [0,         1,           0],
                      [- jnp.sin(theta),     0,        jnp.cos(theta)]])
    z_rot = jnp.array([[ jnp.cos(psi),    - jnp.sin(psi),       0],
                      [ jnp.sin(psi),     jnp.cos(psi),       0],
                      [0,          0,           1]])
    return z_rot @ y_rot @ x_rot

@jit
def R_b__n_inv(phi, theta, psi):
    return jnp.array([[ jnp.cos(psi)* jnp.cos(theta), jnp.cos(theta)* jnp.sin(psi), - jnp.sin(theta)],
                     [ jnp.cos(psi)* jnp.sin(phi)* jnp.sin(theta) - jnp.cos(phi)* jnp.sin(psi), 
                      jnp.cos(phi)* jnp.cos(psi) + jnp.sin(phi)* jnp.sin(psi)* jnp.sin(theta), 
                      jnp.cos(theta)* jnp.sin(phi)],
                     [ jnp.sin(phi)* jnp.sin(psi) + jnp.cos(phi)* jnp.cos(psi)* jnp.sin(theta), 
                      jnp.cos(phi)* jnp.sin(psi)* jnp.sin(theta) - jnp.cos(psi)* jnp.sin(phi), 
                      jnp.cos(phi)* jnp.cos(theta)]])

## Transformation matrix for angular velocity
@jit
def T(phi, theta):
    return jnp.array([[1,     jnp.sin(phi)* jnp.tan(theta),      jnp.cos(phi)* jnp.tan(theta)],
                     [0,          jnp.cos(phi),                   - jnp.sin(phi)],
                     [0,     jnp.sin(phi)/jnp.cos(theta),      jnp.cos(phi)/jnp.cos(theta)]])


Henv = 0.51  # Height of the envelope (m)
Hgon = 0.04  # Height of the gondola (m)
r_z_gb__b = 0.05  # Distance from CG to CB (m)
dvt = Henv / 2 + Hgon # Distance from CB to motor thrust application point
r_z_tg__b = dvt - r_z_gb__b  # Distance from CB to thrust generation point
r_z_gb__b = 0.05
r_gb__b = jnp.array([0, 0, r_z_gb__b]).T

# Mass and inertia properties
m_RB = 0.1249  # Mass of the blimp (kg)
I_RBx = 0.005821  # Rotational inertia about x-axis (kg.m^2)
M_RB_CG = jnp.diag(jnp.array([m_RB, m_RB, m_RB, I_RBx, I_RBx, I_RBx]))# Inertia matrix for the rigid body in body frame




In [13]:

M_RB_CB = H(r_gb__b).T @ M_RB_CG @ H(r_gb__b)
M_A_CB = jnp.diag(jnp.array([0.0466, 0.0466, 0.0545, 0.0, 0.0, 0.0])) # Added mass and inertia matrix (assuming hydrodynamics)

# Total inertia matrix at the center of buoyancy
M_CB = M_RB_CB + M_A_CB
M_CB_inv = jnp.linalg.inv(M_CB)

# Gravitational force in the navigation frame
g_acc = 9.8  # Gravitational acceleration (m/s^2)
fg_n = m_RB * jnp.array([0, 0, g_acc]).T  # Force due to gravity

# Aerodynamic damping matrix
D_CB = jnp.diag(jnp.array([0.0125, 0.0125, 0.0480, 0.000862, 0.000862, 0.000862]))
